## test

In [ ]:
import os

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
#tf.config.experimental_run_functions_eagerly(True)

## import

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image

## feature extractor

In [ ]:
network_name = 'transformers'

In [ ]:
def build_feature_extractor():

    feature_extractor = tf.keras.applications.DenseNet121(weights = "densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                                          include_top = False, pooling = "avg", input_shape = (
        400, 300, 3))
    #preprocess_input  = tf.keras.applications.inception_v3.preprocess_input
    inputs = tf.keras.Input((400, 300, 3))
    #preprocessed      = preprocess_input(inputs)
    outputs = feature_extractor(inputs)

    return tf.keras.Model(inputs, outputs, name = "feature_extractor")


feature_extractor = build_feature_extractor()

In [ ]:
n = os.listdir('I:/poorya/datasets/video dataset/videos lts/')
features = np.zeros((len(n), 40, 1024), dtype = "float32")
strsr_lbl = np.zeros((len(n), 1))
typee_lbl = np.zeros((len(n), 1))

for i in range(len(n)):
    strsr_lbl[i, 0] = int(n[i][-3])
    typee_lbl[i, 0] = int(n[i][-1])
    print(i)

    for j in range(40):
        img = image.img_to_array(image.load_img('I:/poorya/datasets/video dataset/videos lts/' + n[
            i] + '/' + str(j) + '.jpg'))
        img /= 255.0
        features[i, j, :] = feature_extractor.predict(np.expand_dims(img, 0)).squeeze()

In [ ]:
#strsr_lbl = np.load('I:/poorya/datasets/video dataset/lables/strsr_lbl.npy'             ) 
#ll = np.zeros((len(strsr_lbl)))

#for i in range(len(strsr_lbl)) :
#ll[i] = int(np.where(np.array(strsr_lbl[i]) == 1)[0][0])

#ll = ll.astype('int')

In [ ]:
os.mkdir('I:/poorya/datasets/video dataset/' + network_name)
np.save('I:/poorya/datasets/video dataset/' + network_name + '/features.npy', features)
np.save('I:/poorya/datasets/video dataset/lables/strsr_lts_trns_lbl.npy', strsr_lbl)
np.save('I:/poorya/datasets/video dataset/lables/typee_lts_trns_lbl.npy', typee_lbl)
#np.save( 'I:/poorya/datasets/video dataset/lables/ll.npy'                     , ll       )

## data preparation

In [ ]:
#features  = np.load('I:/poorya/datasets/video dataset/' + network_name + '/features.npy')
#typee_lbl = np.load('I:/poorya/datasets/video dataset/lables/typee_lbl.npy'             )
#ll        = np.load('I:/poorya/datasets/video dataset/lables/ll.npy'                    )

In [ ]:
l = len(features)
values = [i for i in range(l)]
permutations = np.random.permutation(values)
X = features[permutations, :, :]
Ys = strsr_lbl[permutations]
Yt = typee_lbl[permutations]

X_train = X[:int(0.8 * l), :, :]
Y_train_s = Ys[:int(0.8 * l)]
Y_train_t = Yt[:int(0.8 * l)]
X_test = X[int(0.8 * l):, :, :]
Y_test_s = Ys[int(0.8 * l):]
Y_test_t = Yt[int(0.8 * l):]

sizee = len(X_test)
val = 0.5
X_val = X_test[:int(val * sizee), :]
Y_val_t = Y_test_t[:int(val * sizee)]
Y_val_s = Y_test_s[:int(val * sizee)]
X_test = X_test[int(val * sizee):, :]
Y_test_t = Y_test_t[int(val * sizee):]
Y_test_s = Y_test_s[int(val * sizee):]

In [ ]:
tr_df = pd.DataFrame({
                         'file name':      [str(i) + '.npy' for i in range(len(X_train))],
                         'stressor lable': [Y_train_s[i] for i in range(len(Y_train_s))],
                         'type lable':     [Y_train_t[i] for i in range(len(Y_train_t))]
                         })

ts_df = pd.DataFrame({
                         'file name':      [str(i) + '.npy' for i in range(len(X_test))],
                         'stressor lable': [Y_test_s[i] for i in range(len(Y_test_s))],
                         'type lable':     [Y_test_t[i] for i in range(len(Y_test_t))]
                         })

vl_df = pd.DataFrame({
                         'file name':      [str(i) + '.npy' for i in range(len(X_val))],
                         'stressor lable': [Y_val_s[i] for i in range(len(Y_val_s))],
                         'type lable':     [Y_val_t[i] for i in range(len(Y_val_t))]
                         })

#'stressor lable': [[int(Y_train[i,0]), int(Y_train[i,1])] for i in range(len(Y_train))]})

In [ ]:
os.mkdir('I:/poorya/datasets/video dataset/' + network_name + '/train')
os.mkdir('I:/poorya/datasets/video dataset/' + network_name + '/test')
os.mkdir('I:/poorya/datasets/video dataset/' + network_name + '/val')

In [ ]:
tr_df.to_csv('I:/poorya/datasets/video dataset/' + network_name + '/train/tr_df.csv')
ts_df.to_csv('I:/poorya/datasets/video dataset/' + network_name + '/test/ts_df.csv')
vl_df.to_csv('I:/poorya/datasets/video dataset/' + network_name + '/val/vl_df.csv')

In [ ]:
def sv(myx, adrs):
    for i in range(len(myx)):
        np.save('I:/poorya/datasets/video dataset/' + network_name + '/' + adrs + str(i) + '.npy', myx[i, :, :])


sv(X_train, 'train/')
sv(X_test, 'test/')
sv(X_val, 'val/')